## Deliverable 2. Create a Customer Travel Destinations Map.

In [110]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [111]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.06,53,100,13.80,overcast clouds
1,1,Kontagora,NG,10.3999,5.4695,76.08,68,33,3.85,scattered clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,61.63,35,9,5.82,clear sky
3,3,Qaanaaq,GL,77.4840,-69.3632,29.14,93,100,4.25,overcast clouds
4,4,Chuy,UY,-33.6971,-53.4616,53.17,94,70,4.90,broken clouds


In [112]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [113]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kontagora,NG,10.3999,5.4695,76.08,68,33,3.85,scattered clouds
9,9,The Valley,AI,18.2170,-63.0578,82.26,79,12,17.65,few clouds
11,11,Waddan,LY,29.1614,16.1390,82.29,22,0,4.18,clear sky
13,13,Georgetown,MY,5.4112,100.3354,82.33,92,20,2.30,few clouds
19,19,Victoria,HK,22.2855,114.1577,81.97,91,100,11.99,overcast clouds
20,20,Hambantota,LK,6.1241,81.1185,77.34,100,20,8.05,few clouds
24,24,Mubi,NG,10.2676,13.2644,75.99,58,97,6.17,overcast clouds
35,35,Gouyave,GD,12.1646,-61.7296,81.37,78,20,23.02,few clouds
40,40,Kavaratti,IN,10.5669,72.6420,82.80,75,100,12.66,overcast clouds
43,43,Nandyal,IN,15.4833,78.4833,85.21,61,63,10.02,broken clouds


In [114]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [115]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()

In [116]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kontagora,NG,76.08,scattered clouds,10.3999,5.4695,
9,The Valley,AI,82.26,few clouds,18.2170,-63.0578,
11,Waddan,LY,82.29,clear sky,29.1614,16.1390,
13,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
19,Victoria,HK,81.97,overcast clouds,22.2855,114.1577,
20,Hambantota,LK,77.34,few clouds,6.1241,81.1185,
24,Mubi,NG,75.99,overcast clouds,10.2676,13.2644,
35,Gouyave,GD,81.37,few clouds,12.1646,-61.7296,
40,Kavaratti,IN,82.80,overcast clouds,10.5669,72.6420,
43,Nandyal,IN,85.21,broken clouds,15.4833,78.4833,


In [117]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [118]:
hotel_df.tail()
hotel_df.count()

City                   184
Country                184
Max Temp               184
Current Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64

In [129]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])

In [131]:
# Check for empty rows
clean_hotel_df.count()

City                   166
Country                166
Max Temp               166
Current Description    166
Lat                    166
Lng                    166
Hotel Name             166
dtype: int64

In [132]:
# 8a. Create the output File (CSV)
vacation_output_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(vacation_output_file, index_label="City_ID")

In [133]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt>{Country}</dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [134]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))